In [3]:
import os
import pandas as pd
from price_loaders.tradingview import load_asset_price
import numpy as np

def save_to_csv(data, folder, filename):
    filepath = os.path.join(folder, filename)
    data.to_csv(filepath, index=False)
    print(f"Lưu dữ liệu thành công vào {filepath}")

In [4]:


# eu_gdp_growth                   = load_asset_price("EUGDPYY", 80, "3M", None) 
# eu_interest_rate                = load_asset_price("EUINTR", 240, "1M", None) 
# eu_inflation_rate               = load_asset_price("EUIRYY", 240, "1M", None)
# eu_consumer_price_index         = load_asset_price("EUCPI", 240, "1M", None)
# eu_producer_price_index         = load_asset_price("EUPPI", 240, "1M", None)
# eu_unemployment_rate            = load_asset_price("EUUR", 240, "1M", None)
# eu_trade_balance                = load_asset_price("EUBOT", 240, "1M", None)
# eu_gov_debt                     = load_asset_price("EUGDG", 10, "12M", None)
# eu_consumer_confidence_index    = load_asset_price("EUCCI", 240, "1M", None)
# eu_retail_sales                 = load_asset_price("EURSMM", 2,"1M", None)
# eu_money_supply                 = load_asset_price("EUM2", 2,"1M", None)



In [45]:
#EURUSD
eurusd_D = load_asset_price("EURUSD", 7560, "1D", None)  # ~30 năm ngày
eurusd_W = load_asset_price("EURUSD", 1560, "1W", None)  # ~30 năm tuần
eurusd_M = load_asset_price("EURUSD", 360, "1M", None)   # ~30 năm tháng

In [6]:
eurusd_D.tail(10)

,time,open,high,low,close,volume
2510,2025-07-07 04:00:00+07:00,1.17777,1.17902,1.16863,1.17080,238564.0
2511,2025-07-08 04:00:00+07:00,1.17080,1.17652,1.16824,1.17246,241445.0
2512,2025-07-09 04:00:00+07:00,1.17246,1.17292,1.16895,1.17198,193830.0
2513,2025-07-10 04:00:00+07:00,1.17198,1.17497,1.16624,1.16998,381604.0
2514,2025-07-11 04:00:00+07:00,1.16998,1.17139,1.16645,1.16883,243393.0
2515,2025-07-14 04:00:00+07:00,1.16638,1.16977,1.16541,1.16641,188178.0
2516,2025-07-15 04:00:00+07:00,1.16641,1.16926,1.15928,1.16009,213759.0
2517,2025-07-16 04:00:00+07:00,1.16009,1.17213,1.15623,1.16399,356498.0
2518,2025-07-17 04:00:00+07:00,1.16399,1.16424,1.15564,1.15950,200336.0
2519,2025-07-18 04:00:00+07:00,1.15950,1.16718,1.15906,1.16251,192096.0


In [46]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

def seasonality_monthly(df, start_year=2015, ignore_months=None):
    """
    Tính seasonal tendency theo tháng từng năm và vẽ heatmap cùng thống kê.

    Parameters:
    - df: DataFrame có 'time' (datetime), 'close' (float)
    - start_year: năm bắt đầu tính toán (bao gồm)
    - ignore_months: list các tháng dạng 'YYYY-MM' để loại bỏ (vd: ['2020-03', '2021-07'])

    Returns:
    - heatmap figure, stats DataFrame
    """
    # Xử lý datetime
    df['time'] = pd.to_datetime(df['time'])
    df = df[df['time'].dt.year >= start_year].copy()

    # Loại bỏ tháng bị ignore nếu có
    if ignore_months:
        ignore_months = set(ignore_months)
        df['year_month'] = df['time'].dt.strftime('%Y-%m')
        df = df[~df['year_month'].isin(ignore_months)]

    # Sắp xếp theo thời gian
    df = df.sort_values('time').reset_index(drop=True)

    # Tính % thay đổi tháng so với tháng trước
    df['pct_change'] = df['close'].pct_change() * 100

    # Tạo cột năm và tháng
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month

    # Tạo ma trận: index là năm, columns là tháng, giá trị là % change
    pivot = df.pivot(index='year', columns='month', values='pct_change')

    # Thống kê từng tháng theo năm: trung bình, độ lệch chuẩn, % positive
    monthly_avg = pivot.mean(axis=0)
    monthly_std = pivot.std(axis=0)
    monthly_pos_pct = (pivot > 0).sum(axis=0) / pivot.count(axis=0) * 100

    # Tạo heatmap
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    fig = go.Figure(data=go.Heatmap(
        z=pivot.values,
        x=month_names,
        y=pivot.index.astype(str),
        colorscale='RdYlGn',
        colorbar=dict(title='% Change'),
        hovertemplate='Year: %{y}<br>Month: %{x}<br>Change: %{z:.2f}%' 
    ))

    # Vẽ thêm các thống kê bên dưới heatmap dưới dạng biểu đồ cột
    fig.add_trace(go.Bar(
        x=month_names,
        y=monthly_avg,
        name='Average % Change',
        marker_color='blue',
        yaxis='y2'
    ))
    fig.add_trace(go.Bar(
        x=month_names,
        y=monthly_std,
        name='Std Dev',
        marker_color='orange',
        yaxis='y2'
    ))
    fig.add_trace(go.Bar(
        x=month_names,
        y=monthly_pos_pct,
        name='% Positive',
        marker_color='green',
        yaxis='y2'
    ))

    # Cấu hình trục y thứ 2 để vẽ thống kê
    fig.update_layout(
        title=f'Seasonal Monthly Changes since {start_year}',
        yaxis=dict(title='Yearly Monthly % Change', domain=[0.3, 1]),
        yaxis2=dict(title='Stats', domain=[0, 0.25], anchor='x'),
        barmode='group',
        height=700,
        legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='right', x=1),
        margin=dict(t=50)
    )

    # Dữ liệu thống kê trả về
    stats_df = pd.DataFrame({
        'Month': month_names,
        'Average % Change': monthly_avg.values,
        'Std Dev': monthly_std.values,
        '% Positive': monthly_pos_pct.values
    })

    return fig, stats_df

# Ví dụ sử dụng:
import pandas as pd

# Giả sử eurusd_D là DataFrame của bạn, đã có cột time dạng datetime với timezone


fig, stats = seasonality_monthly(eurusd_M, start_year=2005)
fig.show()
print(stats)


   Month  Average % Change   Std Dev  % Positive
0    Jan         -0.436774  3.070083   50.000000
1    Feb         -0.343266  1.656905   38.095238
2    Mar          0.594320  2.485492   57.142857
3    Apr          0.852105  2.507922   52.380952
4    May         -0.920917  3.311638   38.095238
5    Jun          0.354620  1.632131   57.142857
6    Jul          0.375020  2.435387   57.142857
7    Aug         -0.301992  1.935027   40.000000
8    Sep         -0.424828  3.218311   40.000000
9    Oct         -0.440356  2.684690   50.000000
10   Nov         -0.292211  2.941251   50.000000
11   Dec          0.790518  3.013761   65.000000


In [65]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def calculate_swing_high_low(df, window=5):
    """
    Tính swing high/low và lọc đỉnh/đáy hợp lệ theo logic đã cho.
    Trả về df với 2 cột mới: 'valid_swing_high', 'valid_swing_low' (bool).
    """
    highs = df['high']
    lows = df['low']

    swing_highs = []
    swing_lows = []
    n = len(df)

    # Tìm điểm swing high/low thô
    for i in range(n):
        start = max(i - window, 0)
        end = min(i + window + 1, n)

        max_around = highs[start:i].max() if i > start else -np.inf
        max_around = max(max_around, highs[i+1:end].max() if i+1 < end else -np.inf)

        min_around = lows[start:i].min() if i > start else np.inf
        min_around = min(min_around, lows[i+1:end].min() if i+1 < end else np.inf)

        swing_highs.append(highs[i] > max_around)
        swing_lows.append(lows[i] < min_around)

    df['swing_high'] = swing_highs
    df['swing_low'] = swing_lows

    swing_high_indices = df.index[df['swing_high']].tolist()
    swing_low_indices = df.index[df['swing_low']].tolist()

    # Hàm kiểm tra swing high hợp lệ
    def is_valid_swing_high(sh_idx):
        lows_before = [idx for idx in swing_low_indices if idx < sh_idx]
        lows_after = [idx for idx in swing_low_indices if idx > sh_idx]
        if not lows_before or not lows_after:
            return False
        low_before_idx = max(lows_before)
        low_after_idx = min(lows_after)
        high_price = df.at[sh_idx, 'high']
        low_before_price = df.at[low_before_idx, 'low']
        low_after_price = df.at[low_after_idx, 'low']
        return low_before_price < high_price and low_after_price < high_price

    # Hàm kiểm tra swing low hợp lệ
    def is_valid_swing_low(sl_idx):
        highs_before = [idx for idx in swing_high_indices if idx < sl_idx]
        highs_after = [idx for idx in swing_high_indices if idx > sl_idx]
        if not highs_before or not highs_after:
            return False
        high_before_idx = max(highs_before)
        high_after_idx = min(highs_after)
        low_price = df.at[sl_idx, 'low']
        high_before_price = df.at[high_before_idx, 'high']
        high_after_price = df.at[high_after_idx, 'high']
        return high_before_price > low_price and high_after_price > low_price

    valid_swing_highs = [(idx, is_valid_swing_high(idx)) for idx in swing_high_indices]
    valid_swing_lows = [(idx, is_valid_swing_low(idx)) for idx in swing_low_indices]

    df['valid_swing_high'] = False
    for idx, valid in valid_swing_highs:
        df.at[idx, 'valid_swing_high'] = valid

    df['valid_swing_low'] = False
    for idx, valid in valid_swing_lows:
        df.at[idx, 'valid_swing_low'] = valid

    # Lọc loại bỏ các đỉnh liên tiếp không hợp lệ
    def filter_consecutive_highs():
        valid_highs = df.index[df['valid_swing_high']].tolist()
        valid_lows = df.index[df['valid_swing_low']].tolist()
        filtered_highs = []

        i = 0
        while i < len(valid_highs):
            curr_high = valid_highs[i]

            if i == len(valid_highs) - 1:
                filtered_highs.append(curr_high)
                break

            next_high = valid_highs[i + 1]
            lows_between = [low for low in valid_lows if curr_high < low < next_high]

            if lows_between:
                filtered_highs.append(curr_high)
                i += 1
            else:
                curr_high_price = df.at[curr_high, 'high']
                next_high_price = df.at[next_high, 'high']

                if curr_high_price > next_high_price:
                    filtered_highs.append(curr_high)
                    i += 2  # bỏ next_high
                else:
                    i += 1  # bỏ curr_high, giữ next_high

        if valid_highs[-1] not in filtered_highs:
            filtered_highs.append(valid_highs[-1])

        df['valid_swing_high'] = False
        df.loc[filtered_highs, 'valid_swing_high'] = True

    # Lọc loại bỏ các đáy liên tiếp không hợp lệ
    def filter_consecutive_lows():
        valid_lows = df.index[df['valid_swing_low']].tolist()
        valid_highs = df.index[df['valid_swing_high']].tolist()
        filtered_lows = []

        i = 0
        while i < len(valid_lows):
            curr_low = valid_lows[i]

            if i == len(valid_lows) - 1:
                filtered_lows.append(curr_low)
                break

            next_low = valid_lows[i + 1]
            highs_between = [high for high in valid_highs if curr_low < high < next_low]

            if highs_between:
                filtered_lows.append(curr_low)
                i += 1
            else:
                curr_low_price = df.at[curr_low, 'low']
                next_low_price = df.at[next_low, 'low']

                if curr_low_price < next_low_price:
                    filtered_lows.append(curr_low)
                    i += 2  # bỏ next_low
                else:
                    i += 1  # bỏ curr_low, giữ next_low

        if valid_lows[-1] not in filtered_lows:
            filtered_lows.append(valid_lows[-1])

        df['valid_swing_low'] = False
        df.loc[filtered_lows, 'valid_swing_low'] = True

    filter_consecutive_highs()
    filter_consecutive_lows()

    return df


def calculate_moving_averages(df, windows=[30, 60, 90]):
    """
    Tính các đường MA với list windows (mặc định 30, 60, 90).
    Trả về df với các cột MA tương ứng.
    """
    for w in windows:
        df[f'MA{w}'] = df['close'].rolling(window=w).mean()
    return df


In [ ]:
# Tính swing high/low hợp lệ
eurusd_D = calculate_swing_high_low(eurusd_D, window=5)

# Tính các đường MA
eurusd_D = calculate_moving_averages(eurusd_D, windows=[30, 60, 90])

eurusd_D['time'] = pd.to_datetime(eurusd_D['time'])
max_date = eurusd_D['time'].max()
start_date = max_date - pd.DateOffset(years=10)
eurusd_D = eurusd_D[eurusd_D['time'] >= start_date].copy()
eurusd_D

,time,open,high,low,close,volume,swing_high,swing_low,valid_swing_high,valid_swing_low,MA30,MA60,MA90
4963,2015-07-20 04:00:00+07:00,1.08299,1.08703,1.08081,1.08244,219608.0,False,True,False,True,NaN,NaN,NaN
4964,2015-07-21 04:00:00+07:00,1.08244,1.09691,1.08116,1.09341,255570.0,False,False,False,False,NaN,NaN,NaN
4965,2015-07-22 04:00:00+07:00,1.09341,1.09667,1.08692,1.09273,251553.0,False,False,False,False,NaN,NaN,NaN
4966,2015-07-23 04:00:00+07:00,1.09273,1.10179,1.09210,1.09818,265263.0,False,False,False,False,NaN,NaN,NaN
4967,2015-07-24 04:00:00+07:00,1.09818,1.09957,1.09253,1.09770,215820.0,False,False,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,2025-07-14 04:00:00+07:00,1.16638,1.16977,1.16541,1.16641,188178.0,False,False,False,False,1.159984,1.145107,1.130119
7556,2025-07-15 04:00:00+07:00,1.16641,1.16926,1.15928,1.16009,213759.0,False,False,False,False,1.160755,1.145409,1.130877
7557,2025-07-16 04:00:00+07:00,1.16009,1.17213,1.15623,1.16399,356498.0,False,False,False,False,1.161499,1.145954,1.131713
7558,2025-07-17 04:00:00+07:00,1.16399,1.16424,1.15564,1.15950,200336.0,False,True,False,False,1.162001,1.146299,1.132540


In [147]:
import plotly.graph_objects as go
import pandas as pd

# Giả sử bạn đã có dataframe eurusd_D với các cột cần thiết

# --- Layout 1: Nến + Volume chung 1 trục x, volume dùng yaxis2 dưới ---

# Vẽ biểu đồ (tuỳ chọn)
fig = go.Figure(data=[go.Candlestick(
    x=eurusd_D['time'],
    open=eurusd_D['open'],
    high=eurusd_D['high'],
    low=eurusd_D['low'],
    close=eurusd_D['close'],
    increasing_line_color='green',
    decreasing_line_color='black'
)])

valid_sh = eurusd_D[eurusd_D['valid_swing_high']]
fig.add_trace(go.Scatter(
    x=valid_sh['time'],
    y=valid_sh['high'],
    mode='markers',
    marker=dict(color='purple', size=10, symbol='star-triangle-up'),
    name='Valid Swing High'
))

valid_sl = eurusd_D[eurusd_D['valid_swing_low']]
fig.add_trace(go.Scatter(
    x=valid_sl['time'],
    y=valid_sl['low'],
    mode='markers',
    marker=dict(color='brown', size=10, symbol='star-triangle-down'),
    name='Valid Swing Low'
))

# Thêm các đường MA
fig.add_trace(go.Scatter(
    x=eurusd_D['time'], y=eurusd_D['MA30'],
    mode='lines', line=dict(color='blue', width=1.5),
    name='MA30'
))
fig.add_trace(go.Scatter(
    x=eurusd_D['time'], y=eurusd_D['MA60'],
    mode='lines', line=dict(color='orange', width=1.5),
    name='MA60'
))
fig.add_trace(go.Scatter(
    x=eurusd_D['time'], y=eurusd_D['MA90'],
    mode='lines', line=dict(color='green', width=1.5),
    name='MA90'
))


volume_colors = ['green' if c >= o else 'black' for o, c in zip(eurusd_D['open'], eurusd_D['close'])]

fig.add_trace(go.Bar(
    x=eurusd_D['time'],
    y=eurusd_D['volume'],
    name='Volume',
    marker_color=volume_colors,
    yaxis='y2',
    opacity=0.6
))

# Cấu hình layout với 2 trục y
fig.update_layout(
    title='EURUSD - Daily Candlestick',
    xaxis=dict(
        domain=[0, 1],
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1Y", step="year", stepmode="backward"),
                dict(count=2, label="2y", step="year", stepmode="backward"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(count=10, label="10y", step="year", stepmode="backward"),
                dict(count=20, label="20y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    ),
    yaxis=dict(
        title='Price',
        domain=[0.3, 1],  # phần trên dành cho giá
        autorange=True
    ),
    yaxis2=dict(
        title='Volume',
        domain=[0, 0.25],  # phần dưới dành cho volume
        autorange=True,
        showgrid=False
    ),
    xaxis_rangeslider_visible=False,
    template='plotly_white',
    height=700,
    width=1200,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()



# --- Layout 2: Seasonal chart riêng biệt ---
def monthly_avg_multi_periods(df, max_year, start_years):
    df = df.copy()
    df['time'] = pd.to_datetime(df['time'])
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month

    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    all_data = []

    for start_year in start_years:
        temp_df = df[df['year'] >= start_year].copy()
        temp_df = temp_df.sort_values('time').reset_index(drop=True)
        temp_df['pct_change'] = temp_df['close'].pct_change() * 100
        pivot = temp_df.pivot(index='year', columns='month', values='pct_change')
        monthly_avg = pivot.mean(axis=0)
        
        for month_idx, avg_val in monthly_avg.items():
            all_data.append({
                'Month': month_names[month_idx - 1],
                'Average % Change': avg_val,
                'Period': f'{max_year - start_year} - Year'
            })

    df_all = pd.DataFrame(all_data)

    fig_line = px.line(df_all, x='Month', y='Average % Change', color='Period',
                       title='Seasonal Tendency Charts',
                       markers=True,
                       labels={'Average % Change': '% Change'})
    fig_line.update_layout(yaxis_tickformat='.2f',
                           template='plotly_white',
                           height=400,
                           width=1200,
                           legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

    return fig_line


# --- Ví dụ sử dụng ---
max_year = eurusd_M['time'].dt.year.max()

start_years = [
    max_year - 20,
    max_year - 10,
    max_year - 5,
    max_year - 2, 
]

fig = monthly_avg_multi_periods(eurusd_M, max_year ,start_years)
fig.show()

